# **=============== FUTURE SALES FORECAST ==============**

Pada project kali ini, saya akan mencoba untuk mempredikis penjualan atau total sales setiap produk dan toko pada setiap bulanya dengan menggunakan Time Series Analysis (ARIMA, SARIMA dan PROPHET). Seperti biasa, pada notebook ini akan dibagi kedalam beberapa bagian:

1. Konteks
2. Data Preprocessing
3. Modelling 
4. Kesimpulan

Pada notebook ini tidak akan secara mendalam membahas teorinya, namun saya akan menjelaskan kenapa langkah-langkah itu yang diambil. Mari kita mulai!

# **1. KONTEKS**

Project ini merupakan salah satu dari sebuah kompetisi di Kaggle. Datasetnya didapat dari penjualan produk pada perusahan di Rusia bernama 1C Company.

* Tujuan: memprediksi penjualan / sales bulan selanjutnya
* Data: 

    1. sales_train.csv (training set), data historis harian mulai dari Januari 2013 sampai Oktober 2015.
    2. test.csv (test set) kita harus memprediksi set ini yaitu penjualan toko dan produk pada bulan November 2015.
    3. sample_submission.csv - contoh hasil untuk dimasukkan ke dalam perlombaan
    4. items.csv - informasi tambahan terkait item / produk
    5. item_categories.csv - informasi tambahan terkait kategori item / produks 
    6. shops.csv- informasi tambahan terkait toko
    
 
* Detail Data:

    1. ID - sebuah id untuk  toko dan item
    2. shop_id - unique identifier untuk toko
    3. item_id - unique identifier item / produk
    4. item_category_id - unique identifier dari kategori itemnya
    5. item_cnt_day - jumlah item yang terjual harian, namun kita akan memprediksi ini dalam bulanan (November 2015)
    6. item_price - harga sebuah itemm
    7. date - tangal
    8. date_block_num - integer dari bulan pada kolom date untuk memudahkan, contoh: Januari 2013 itu 0, Februari 2013 itu 1, Oktober 2015 itu 33, dst
    9. item_name - nama item / produk
    10. shop_name - nama toko
    11. item_category_name - nama kategori
    

* Metriks Evaluasi: Root Mean Squared Error (RMSE)

* Manfaat: ada banyak sekali manfaat yang dapat diberikan dari prediksi ini, anatara lain,

    1. Marketing: membuat promo berdasarkan season yang ada pada waktu sebuah item paling laku
    2. Logistik: menyiapkan proses pengiriman agar tepat waktu
    3. Sales: meningkatkan pendapatan dari produk-produk yang paling laku, untuk lebih dimaksimalkan Harga Pokok Produknya
    4. Produksi: menyiapkan item sesuai dengan prediksi agar tidak terjadi kekurangan item saat high season atau dead stock saat low season

In [1]:
#Import packages /libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from math import sqrt

#Modeling
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from fbprophet import Prophet

#Evaluation
from sklearn.metrics import mean_squared_error
from fbprophet.diagnostics import performance_metrics
from fbprophet.diagnostics import cross_validation
from fbprophet.plot import plot_cross_validation_metric

#Menghilangkan warning
import warnings
warnings.filterwarnings("ignore")

In [1]:
#Melihat data yang ada di directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
#Load semua data
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
sales = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
submission = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv') #ini kita tidak pakai, karena hanya untuk input kompetisi
category = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')

# **2. DATA PREPROCESSING**


Pada tahapan ini, kita akan mencoba untuk membersikan data sebelum masuk ke dalam model dan juga melakukan Exploratory Data Analysis untuk mendapatkan info-info yang menarik dan berguna untuk perusahaan.

## 2.1 Handling Variable

Melihat apakah input data dengan tipe data sudah sesuai atau belum, melakukan sedikit transformasi jika diperlukan atau aggregasi/grouping untuk memudahkan pada proses selanjutnya

In [1]:
#Mengabungkan semua data
data_1 = pd.merge(items, sales, on='item_id', how='inner')
data_2 = pd.merge(data_1, shops, on='shop_id', how='inner')
data = pd.merge(data_2, category, on='item_category_id', how='inner')

In [1]:
#Melihat sekilas data yang telah digabung
data.head()

In [1]:
#Melihat info pada data 
data.info()

* Terdapat 10 kolom, 2935849 input, tipe data yang kita punya float64 ada 2, int64 ada 4, dan object ada 4.

In [1]:
#Merubah tipe data kolom item_cnt_day menjadi integer, karena inputnya adalah berapa barang yang terjual
data['item_cnt_day'] = data.item_cnt_day.astype('int')

In [1]:
#Merubah kolom date menjadi index dan merubah tipe datanya menjadi date
data['date'] = pd.to_datetime(data.date)
data =  data.sort_values('date').reset_index(drop=True)

In [1]:
#Melihat kembali dataset kita apakah kolom date sudah sesuai
data.head()

In [1]:
#Melihat apakah ada input yang kosong pada data kita
data.isnull().sum() /len(data) *100

Ternyata dataset yang kita miliki tidak ada input yang kosong,sehingga kita tidak perlu melakukan pengisian

## 2.2 Feature Engineering

Melakuakn penambahan variabel pada data untuk memaksimalkan model ataupun untuk dapat mencari info-info yang menarik dari data

In [1]:
#Membuat kolom total_sales untuk melihat revenue / pendapatan perhari
data['total_sales'] = data['item_price'] * data['item_cnt_day']
data.head()

## 2.3 Handling Outliers

Pada tahapan ini kita mencoba mencari anomali atau outliers pada data yang kita miliki dan menilai apakah outliers ini memang kesalah, wajar atau menag acak

In [1]:
#Kita akan menggunakan Inter Quartile Range untuk menangani outliers
#Menentukan Limit
def limit(i):
    Q1,Q3 = np.percentile(data[i] , [25,75])
    IQR = Q3 - Q1
    
    #menentukan upper limit biasa dan upper limit ekstim
    lower_limit = Q1 - (IQR * 1.5)
    lower_limit_extreme = Q1 - (IQR * 3)
    upper_limit = Q3 + (IQR * 1.5)
    upper_limit_extreme = Q3 + (IQR * 3)
    print('Lower Limit:', lower_limit)
    print('Lower Limit Extreme:', lower_limit_extreme)
    print('Upper Limit:', upper_limit)
    print('Upper Limit Extreme:', upper_limit_extreme)

#Mengitung persen outliers dari data    
def percent_outliers(i):
    Q1,Q3 = np.percentile(data[i] , [25,75])
    IQR = Q3 - Q1
    
    #menentukan upper limit biasa dan upper limit ekstim
    lower_limit = Q1 - (IQR * 1.5)
    lower_limit_extreme = Q1 - (IQR * 3)
    upper_limit = Q3 + (IQR * 1.5)
    upper_limit_extreme = Q3 + (IQR * 3)
    #melihat persenan outliers terhadap total data
    print('Lower Limit: {} %'.format(data[(data[i] >= lower_limit)].shape[0]/ data.shape[0]*100))
    print('Lower Limit Extereme: {} %'.format(data[(data[i] >= lower_limit_extreme)].shape[0]/data.shape[0]*100))
    print('Upper Limit: {} %'.format(data[(data[i] >= upper_limit)].shape[0]/ data.shape[0]*100))
    print('Upper Limit Extereme: {} %'.format(data[(data[i] >= upper_limit_extreme)].shape[0]/data.shape[0]*100))

In [1]:
#Melihat outliers pada kolom item_price
sns.boxplot(x=data["item_price"])

In [1]:
#Melihat apakah ada harga item yang 0 atau minus. Karena jika ada ini merupakan sebuah kesalahan
data[data['item_price'] <= 0].count()

Ternyata ada 1 item yang harganya 0 atau dibawah 0, maka kita akan hilangkan data tersebutkarena tidak mungkin ada harga jual 0 atau bahkan minus

In [1]:
#Menghilangkan input yang <= 0
data = data[data['item_price'] > 0]

In [1]:
#Melihat IQR dari kolom item_price
print(limit('item_price'))
print('-'*50)
print(percent_outliers('item_price'))

Saya tidak mengaggap yang melebihi upper limit / upper limit extreme adalah outlier karena toko yang ada pada dataset ini memiliki variasi kategori item, sehingga mungkin saja salah satu item memiliki harga yang sangat tinggi atau sangat rendah, sehingga saja tidak menganggapnya outliers. Perlu diketahui juga bahwa dataset ini merupakan kumpulan dari beberapa toko dan kategori item, sehingga sangat memungkinkan terdapat harganya yang sangat beda.

In [1]:
#Melihat outliers pada kolom item_cnt_day
sns.boxplot(x=data["item_cnt_day"])

In [1]:
#Melihat IQR pada kolom itm_cnt_day
print(limit('item_cnt_day'))
print('-'*50)
print(percent_outliers('item_cnt_day'))

Sebenarnya pada kolom item_cnt_day terdapat outlier yang inputnya 0 ataupun minus, ini akan kita cek di bawah dan menghilangkanya karena tidak mungkin ada penjualan yang 0(karena kita hanya akan mengambil yang laku) dan dibawah 0. Namun selain itu menurut saya bahwa mungkin saja dalam satu hari terdapat banyak sekali order dalam satu item karena mungkin ada hari-hari tertentu seperti natal, hari ibu dll yang biasanya banyak diskon yang mengakibatkan meningkatnya orderan, sehingga saya tidak menganggapnya sebagai outliers

In [1]:
#Mengecek apakah ada item yang dijual dalam jumlah 0 atau kurangdari 0, karena jika ada itu merupakan kesalahan
#Karena tida kmungkin item dijual dengan jumlah 0 (karena kita hanya mengambil yang laku) atau bahkan minus
data[data['item_cnt_day'] <= 0].count()

Seperti kita lihat di atas, ternyata ada 7356 input yang salah, sehingga kita akan hilangkan saja karena tidak mungkin kita gantikan secara statistik (mean/median)

In [1]:
#Menghilangkan input yang <= 0
data = data[data['item_cnt_day'] > 0]

## 2.4 Exploratory Data Analysis

Sebelum masuk kedalama model, kita akan mencoba untuk mencari info-info apa yang menarik dari data yang kita miliki sperti item apa yang paling laku, toko mana yang paling laku, item apa yang paling mahal, toko mana yang pendapanya paling banyak, item apa yang paling tidak laku, dll. Mari kita cari info-info menarik!

## **======================== 10 Item Terlaris ==========================**

In [1]:
#Produk apa yang paling laris?
top_10_product_best_seller = data.groupby(['item_name'])['item_cnt_day'].sum().sort_values(ascending=False)[:10]

#Visualisasi 
plt.figure(figsize=(16,9))
sns.barplot(y=top_10_product_best_seller.index,x=top_10_product_best_seller.values)
plt.title('Top 10 Most Selling Items',fontsize=20)
plt.xlabel('Total Product Sold',fontsize=17)
plt.ylabel('Item Name',fontsize=17)

In [1]:
top_10_product_best_seller

Bisa kita lihat, bahwa yang paling banyak penjualnya dalam hal jumlah adalah "Фирменный пакет майка 1С Интерес белый (34*42) 45 мкм" sebanyak 187666. Jika dilihat di google ini adalah kantong kresek, jadi wajar saja jika dia adalah paling bannyak penjualnya dan juga perlu diingat kembali bahwa ini adalah data dari dari Januari 2013 sampai OKtober 2015. jadi hampir 3 tahun. Kemudian jika dilihat di bawahnya kebanyakan adalah video game dan juga ada Playstation 4.

## **======================== 10 Item Paling Tidak Laku ==========================**

In [1]:
#Produk apa yang paling tidak laris?
top_10_product_least_seller = data.groupby(['item_name'])['item_cnt_day'].sum().sort_values(ascending=True)[:10]

#Visualisasi 
plt.figure(figsize=(16,9))
sns.barplot(y=top_10_product_least_seller.index,x=top_10_product_least_seller.values)
plt.title('Top 10 Least Selling Items',fontsize=20)
plt.xlabel('Total Product Sold',fontsize=17)
plt.ylabel('Item Name',fontsize=17)

Ini adalah 10 item yang paling tidak laku, penjualanya hanya 1 dalam waktu 2 tahun 10 bulan

## **======================== 10 Toko Terlaris ==========================**

In [1]:
#Toko apa yang paling laris?
top_10_seller = data.groupby(['shop_name'])['item_cnt_day'].sum().sort_values(ascending=False)[:10]

#Visualisasi 
plt.figure(figsize=(16,9))
sns.barplot(y=top_10_seller.index,x=top_10_seller.values)
plt.title('Top 10 Most Selling Shop',fontsize=20)
plt.xlabel('Total Product Sold',fontsize=17)
plt.ylabel('Shop Name',fontsize=17)

In [1]:
top_10_seller

Toko terbanyak dalma penjualan item adalah "Москва ТЦ "Семеновский" " dengan total penjualan 311230 item. Jika dilihat di google artinya adalah Semenovskiy Shopping & Entertainment Center, toko ini adalah Mall yang menjual segala macam kebutuhan, mulai dari fashion, gadget sampai makanan, sehingga wajar jika penjualanya no satu.

## **======================== 10 Toko Tidak Laris ==========================**

In [1]:
#Toko apa yang paling tidak laris?
top_10_least_seller = data.groupby(['shop_name'])['item_cnt_day'].sum().sort_values(ascending=True)[:10]

#Visualisasi 
plt.figure(figsize=(16,9))
sns.barplot(y=top_10_least_seller.index,x=top_10_least_seller.values)
plt.title('Top 10 Least Shop',fontsize=20)
plt.xlabel('Total Product Sold',fontsize=17)
plt.ylabel('Shop Name',fontsize=17)

In [1]:
top_10_least_seller

Seperti kita lihat bahwa toko yang paling tidak laku adalah Новосибирск ТРЦ "Галерея Новосибирск" dengan 333 item terjual dalam waktu 2 tahun 10 bulan atau sekitar 9-10 item perbulan.

## **====================== 10 Toko Pendapatan Paling Tinggi ======================**

In [1]:
#Toko apa yang paling laris?
top_10_seller = data.groupby(['shop_name'])['total_sales'].sum().sort_values(ascending=False)[:10]

#Visualisasi
plt.figure(figsize=(16,9))
sns.barplot(y=top_10_seller.index,x=top_10_seller.values)
plt.title('Top 10 Shop Revenue',fontsize=20)
plt.xlabel('Total Revenue',fontsize=17)
plt.ylabel('Shop Name',fontsize=17)

In [1]:
top_10_seller

Toko tertinggi pendapatanya adalah Москва ТЦ "Семеновский" dengan 2.356611e+08 Rubel atau setara 48.431.364.028,28 (48 Miliar). Jika kita lihat di google, memang jika diartikan adalah ini sebuah Mall yang berada di Rusia,jadi wajar saja jika keuntunganya sebesar itu, karena pendapatanya tidak hanya dari satu kategori item.

## **====================== 10 Toko Pendapatan Paling Rendah ======================**

In [1]:
#Toko apa yang paling laris?
top_10_seller = data.groupby(['shop_name'])['total_sales'].sum().sort_values(ascending=True)[:10]

#Visualisasi
plt.figure(figsize=(16,9))
sns.barplot(y=top_10_seller.index,x=top_10_seller.values)
plt.title('Top 10 Least Shop Revenuw',fontsize=20)
plt.xlabel('Total Revenue',fontsize=17)
plt.ylabel('Shop Name',fontsize=17)

In [1]:
top_10_seller

Toko yang memiliki pendapatan paling rendah adalah Новосибирск ТРЦ "Галерея Новосибирск" dengan 379461.00 Rubel sekitar 77.984.078,94 (77 Juta). Toko ini juga merupakan seperti Mall / marketplace.

## **======================== 10 Kategori Terlaris ==========================**

In [1]:
#kategori apa yang paling laris?
top_10_most_category = data.groupby(['item_category_name'])['item_cnt_day'].sum().sort_values(ascending=False)[:10]

#Visualisasi 10 item paling laris
plt.figure(figsize=(16,9))
sns.barplot(y=top_10_most_category.index,x=top_10_most_category.values)
plt.title('Top 10 Most Selling Category',fontsize=20)
plt.xlabel('Total Product Sold',fontsize=17)
plt.ylabel('Category Name',fontsize=17)

Seperti yang sudah diperkirakan,karena 10 produk terbanyak dalam penjualan adalah kebanyakan entertainment, maka tidak heran jika kategori produk yang paling laku juga enternainment seperti pada urutan paling atas adalah kategori DVD, kemudian adajuga PS 3, PS 4 dan XBOX 360.

## **======================== 10 Kategori Paling Tidak Laku ==========================**

In [1]:
#kategori apa yang paling tidak laris?
top_10_least_category = data.groupby(['item_category_name'])['item_cnt_day'].sum().sort_values(ascending=True)[:10]

#Visualisasi 10 item paling laris
plt.figure(figsize=(16,9))
sns.barplot(y=top_10_least_category.index,x=top_10_least_category.values)
plt.title('Top 10 Least Selling Category',fontsize=20)
plt.xlabel('Total Product Sold',fontsize=17)
plt.ylabel('Category Name',fontsize=17)

kategori yang paling sedikit penjualanya adalah Книги - Познавательная литература jika diartikan maka akan muncul buku, dan kebanyakn dari 10 ini adalah buku semua, mungkin memang kategori buku kurang diminati.

# **3. MODELING**

Pada tahapan ini kita akan mencoba membuat model Time Series dengan menggunakan ARIMA, SARIMA dan PROPHET

In [1]:
#Merubah tipe data kolom date menjadi tipe date
data =  data.set_index('date')
data.head()

In [1]:
#Membuat data menjadi sebuah group dengan menghitung total sales setiap bulanya
train_arima = data.resample("M").sum() 
ts_sales = train_arima[["total_sales"]]
ts_sales.head()

In [1]:
#Menghilangkan data setelah oktober 2015 karena data setelah oktober tidak relevan, hanya sedikit data yang diambil
ts_sales = ts_sales[ts_sales.index <= pd.to_datetime('2015-10-31')]

## **3.1 Stationary**

Time series analysis membutuhkan data yang stationary, yaitu ketika mean, variance, dan covariance dianggap konstan sepanjang waktu agar mudah untuk dilakukan prediksinya, karena akan terlihat polanya. Untuk mengetahui apakah suatu data stationary atau tidak, disini saya menggunakan Dickey-Fuller test. Jika p-value di atas 0.05 maka datanya tidak stationary, namun jika di bawah 0.05 maka stationary.

In [1]:
#Viasualisasi data
plt.figure(figsize=(16,9))
plt.title('Total Item of the company')
plt.xlabel('Month')
plt.ylabel('Item')
plt.plot(ts_sales['total_sales'])

Sebenaryya dari grafik di atas kita sudah bisa melihat bahwa data yang kita miliki kemungkinan stasionary,karena tidak ada perubahan dari mean, variance atau covariance secara kasat mata, namun kita perlu cek dengan Dickey-Fuller test.

In [1]:
#Melihat apakah data kita stationary atau tidak dengan Dickey-Fuller Test
#Membuat fungsi
def test_stationarity(timeseries):
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [1]:
#Cek Stationary
test_stationarity(ts_sales['total_sales'])

Seperti yang kita lihat di atas bahwa data memiliki p-value 0.000484 maka datanya stationary, sehingga kita tidak perlu membuatnya stationary dengan melakukan trnasformasi dengan differencing

## **3.2 Trend& Seasonality**

Untuk melihat apakah sebuah data memiliki trend yang cenderung naik atau turun dan juga melihat apakah ada siklus yang berulang pada waktu-waktu tertentu, contoh ketika setiap sebelum lebaran banyak orang yang belanja sehingga setiap waktu itu akan terjadi peningkatan penjualan. 

In [1]:
#Visualisasi Trend dan Seasonlaity
item_cnt_dec = sm.tsa.seasonal_decompose(ts_sales['total_sales'],freq=12).plot()

Darai kedua visualisasi di atas kita dapat melihat:
- data ini memiliki seasonality, terjadi peningkatan pada setiap akhir tahun serta penurunan di awal tahun
- terjadi peningkatan trend di awal sampaipertengahan data dan kemudian mengalami penuruan

## **3.3 Autocorrelation**

Autocorrelation digunakan untuk melihat apakah ada hubungan / korelasi antara nilai sekarang dengan nilai sebelumnya (lag / lagging / selisih). Setiap model regressi, mengasumsikan bahwa data tida ada korelasi. Autocorrelasi juga bisa digunakna untuk melihat seasonality atau trend yang tidak terlihat pada Lag 1, 2 dst.

In [1]:
#Visualisasi autocorrelation plot pada data item_cnt_day
sales_acf = sm.graphics.tsa.plot_acf(ts_sales['total_sales'], lags=12)

In [1]:
#Visualisasi autocorrelation plot pada data item_cnt_day
sales_acf = sm.graphics.tsa.plot_pacf(ts_sales['total_sales'], lags=12)

kita dapat melihat bahwa 95% data kita masuk kedalam confidence interval (area warna biru) sehingga bisa dikatakan data kita tidak memiliki autocorrelation. Kemudian dari data di atas bisa kita lihat bahwa nilai p dan q ada pada 1 atau 2

## **3.4 Modelling (SARIMA)**


Pada tahapan ini saya akan membuat prediksi dengan SARIMA (Seasonality Autoreggesive Integrated Moving Average) sebernarnya ini 4 metode yang dijadikan satu. Terdapat 4 parameter yaitu:
 - p: Seasonality (Autoregressive / AR)
 - d: Trend (Integrated / I)
 - q: Noice (Moving Average / MA)
 - s: Seasonality

In [1]:
#Membuat nilai p,d,q dengan rentang 0 and 3
p = d = q = range(0, 2)

#Membuat iterasi nilai p,dq
pdq = list(itertools.product(p, d, q))


#Membuat seasonal dengan 12 bulan (karena yang paling terlihat / smooth)
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [1]:
#Melakukan pencarian nilai p,d,q dengan parameter AIC, semakin rendah semakin baik
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(ts_sales['total_sales'],
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=True,
                                            enforce_invertibility=True)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

Dari pencarian di atas, kita mendapatkan paramter p,d,q terbaik adalah ARIMA(0, 1, 0)x(0, 1, 1, 12)12 - AIC:755.0370288027423 . Sekarang akan kita masukkan ke dalam model.

In [1]:
#Kita masukkan kedalma model
mod = sm.tsa.statespace.SARIMAX(ts_sales['total_sales'],
                                order=(0,1,0),
                                seasonal_order=(0, 1, 1, 12),
                                enforce_stationarity=True,
                                enforce_invertibility=True)
results = mod.fit()
print(results.summary().tables[1])

In [1]:
results.plot_diagnostics(figsize=(16, 9))
plt.show()

Dari graifk residual (Normal Q-Q), Corrleogram / ACF PCF, histogram kde, menunjukkan bahwa model yang kita buat sudah cukup optimal dengan melihat persebaranya yang mendekati normal

In [1]:
#Melakukan prediksi dari bulan Desember 2014
pred = results.get_prediction(start=pd.to_datetime('2014-12-31'), dynamic=False)
pred_ci = pred.conf_int()

In [1]:
#Visualisasi prediksi
ax = ts_sales['2013-01-31':].plot(label = "observed", figsize=(16, 9))
pred.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Month')
ax.set_ylabel('Sales')
plt.legend()
plt.show()

train_sarima_forecasted = pred.predicted_mean
train_sarima_truth = ts_sales['2014-12-31':]

#Menghiung RMSE
rmse_sarima = sqrt(mean_squared_error(train_sarima_truth, train_sarima_forecasted))
print("Root Mean Squared Error: ", rmse_sarima)

Pada model SARIMA kita mendapatkan RMSE sebesar 12688719.01 yang artinya kemungkinan eror data kita dengan predikisinya seberar tersebut. Saya rasa ini sudah lumayan optimal,karena perbandingan dari besaran total sales reratanya mencapai 98 juta atau kemungkinan model kita salah prediksi dari nilai sebesar 12.8% dari nilai.

In [1]:
#Prediksi 3 total sales 3 bulan kedepan
pred_uc = results.get_forecast(steps=3)
pred_ci = pred_uc.conf_int()
ax = ts_sales['2013-01-31':].plot(label='observed', figsize=(16, 9))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Amount of Sales')
plt.legend()
plt.show()

Seperti yang kita lihat bahwa prediksinya memang sesuia dugaan, yaitu pada akhir tahun memang akan terjadi peningkatan sales dan pada wal tahun akan terjadi penurunan sales.

## **3.5 Modelling (PROPHET)**

Prophet adalah library yang dikeluarkan oleh facebook untuk melakukan Time Series Analysis, perbedaanya dengan yang lain adalah lebih ke praktisan dan ke akuratanya. Krena kita sudah mengecek tentang Stationary, Seasonality, Autocorrelation dll pada model sebelumnya, maka sekarang langsung melakukan model.

In [1]:
#Reset index dulu untuk membuat date menjadi kolom
ts_sales = ts_sales.reset_index()
#Pertama kita harus merubah dates menjadi ds dan total_sales menjadi y
ts_sales.rename(columns={'date':'ds','total_sales':'y'},inplace=True)

In [1]:
#Membuat prediksi dengan Prophet
m_p = Prophet()
m_p.fit(ts_sales)
future = m_p.make_future_dataframe(periods = 3, freq = 'M')
prediction = m_p.predict(future)
prediction.tail(3)

In [1]:
#Visualisai Prediksi
m_p.plot(prediction)
plt.show()

Model dengan menggunakan Prophet bisa kita lihat cukup baik, titik yang berwarna hitam adalah nilai aktualnya dan garis yang berwarna biru adalah nilai prediksinya, bisa dilihat bahwa model kita sudah cukup baik dalam memprediksi. Seperti pada prediksi dengan menggunakan SARIMA, prediksi yang dihasilkan sama-sama menunjukkan bahwa terjadi peningkatan pada bulan November sampai Januari dan kemudian mengalami penurunan setelah bulan Januari.

In [1]:
#Visualisasi [Trends,Weekly]
m_p.plot_components(prediction)
plt.show()

Seperti yang kita lihat di atas, trend menunjukkan penurunan, hal ini sesuai dengan trend yang memang dari awal dataset ini perlihatkan seperti pada model SARIMA sebelumnya

## Crossvalidation

Melihat eror dari nilai yang diprediksi dengan nilai aslinya dengan meggunakan parameter:
- initial:berapa banyak data yang digunakan untuk training
- period : berapa banyak yang digunakan untuk prediksi
- horizon : berapa rentang waktu yang digunakan untuk prediksi
- Secara defalut, initial traing itu 3x dari horizon dan periode itu setengahnya horizon

In [1]:
#Melakukan Crossvalidation dengan training data 720 (2 tahun), prediksi 120 (4 bulan) dan interval waktunya 240 (8 bulan)
cv = cross_validation(m_p,initial='720 days', period='120 days', horizon = '240 days')

In [1]:
#Melihat hasil cv
cv.head()

In [1]:
#Melihat performance metrics
df_pm= performance_metrics(cv)
df_pm.head()

In [1]:
#Visualiasi RMSE
plot_cross_validation_metric(cv, metric='rmse')
plt.show()

Jika kita rerata rmse dari model prophet maka didapatkan 23.726.608.27, sehingga model  ini lebih buruk dari SARIMA sebelumnya,hal ini mungkin bisa dilakukan tunning, dengan menambahkan holiday, dll karena prophet sangat bekerja baik jika data yang dimasukkan sangat detail.

# **4. KESIMPULAN**


1. Model terbaik pada project kali ini adalah dengan menggunakan SARIMA
2. RMSE yang didapat 12.688.719,01 atau 12.8% dari rata-rata data
3. Perlu adanya tunning untuk model Prophet dengan menambahkan variabel lain seperti holiday
4. Prediksi total sales 3 bulan kedepan seperti di bawah:

In [1]:
pred_ci